In [1]:
# Import Libraries and Dependencies
import pandas as pd

### Read in files

In [2]:
# Read in data and index by CustomerID


In [3]:
# Show sample of January sales data.


In [4]:
# Show sample of February sales data.


In [5]:
# Show sample of March sales data.


### Check the data types of each DataFrame

In [6]:
# Jan sales datatypes

In [7]:
# Feb sales datatypes

In [8]:
# March sales datatypes

### Combine the sales data by rows.

In [9]:
# Combine the January, February, and March sales DataFrames on the rows using an inner join, and reset the index


In [10]:
# Check if any values are null.


In [11]:
# Check the data type of each column


In [12]:
# Convert the "order_date" column to a datetime datatype.


In [13]:
# Confirm that the data type has been changed.


### 1. Which top five zip codes in which cities and states had the greatest number of products ordered?

### Using `groupby`

In [14]:
# Show the average number items ordered for each state, city, and zip code.
# Rename the sum column to reflect the data in the column.



# Show the top 5 results.


### Using `pivot_table`

In [15]:
# Show the total number of items ordered for each state, city, and zip code.


# Rename the "sum" column to "Avg_Number_Products_Ordered"


# Show the top 5 results.


### 2. Which top five zip codes in which cities and states generated the most sales?

### Using `groupby`

In [16]:
# Show the total price for the items ordered for each state, city, and zip code.
# Rename the sum column to reflect the data in the column.


# Show the top 5 results.


### Using `pivot_table`

In [17]:
# Show the total price for all of the items ordered for each state, city, and zip code.


# Rename the "price" column to "Avg_Number_Products_Ordered"

# Show the top 5 results.


### 3. Which top five zip codes in which cities and states had the greatest average price for the products ordered?

### Using `groupby`

In [18]:
# Show the average price of items ordered for each state, city, and zip code rounded to two decimal places.
# Rename the mean column to reflect the data in the column.


# Show the top 5 results.


### Using `pivot_table`

In [19]:
# Show the average price per product ordered for each state, city, and zip code rounded to two decimal places.


# Rename the "price" column to "Avg_Price"


# Show the top 5 results.


### 4. Which top five zip codes in which cities and states had the most iPhone sales?

In [20]:
# Filter the sales data to get the iPhone sales data.


### Using `groupby`

In [21]:
# Show the total number of items ordered for each state, city, and zip code.
# Rename the count column to reflect the data in the column.

# Show the top 5 results.


### Using `pivot_table`

In [22]:
# Show the total number of items ordered for each city and zip code.

# Rename the "quantity_ordered" column to "Number_Ordered"


# Show the top 5 results.


### 5. Which day had the most iPhone sales?

In [23]:
# Create a pivot table with the 'order_date' as the index and the "price" as the values.


# Rename the "price" column to "Total_Sales"


# Show the table.


In [24]:
# Resample the pivot table into daily bins 
# and get the total sales for each day rounded to two decimal places.


# Sort the resampled pivot table in ascending order on "Total_Sales".


### 6.  Which week had the most iPhone sales?

In [25]:
# Resample the pivot table into weekly bins 
# and get the total sales for each week rounded to two decimal places.


# Sort the resampled pivot table in ascending order on "Total_Sales".
